In [7]:
import numpy as np

def generate_json_per_type(mask, spacing_mm, class_labels):
    json_result = {}
    
    type_counts = {label: np.count_nonzero(mask == i+1) for i, label in enumerate(class_labels)}
    print(type_counts)
    type_counts = {k: v for k, v in type_counts.items() if v > 0}

    json_result["hemorrhage_detected"] = len(type_counts) > 0
    json_result["types"] = []

    for i, label in enumerate(class_labels):
        voxels = np.count_nonzero(mask == i+1)
        if voxels == 0:
            continue

        voxel_volume_mm3 = spacing_mm[0] * spacing_mm[1] * spacing_mm[2]
        volume_cm3 = voxels * voxel_volume_mm3 / 1000  # mm3 -> cm3

        coords = np.array(np.nonzero(mask == i+1))
        if coords.size > 0:
            min_coords = coords.min(axis=1)
            max_coords = coords.max(axis=1)
            size_mm = (max_coords - min_coords + 1) * np.array(spacing_mm)
            size_str = "x".join([str(int(s)) for s in size_mm])
        else:
            size_str = "0x0x0"

        json_result["types"].append({
            "type": label,
            "volume_cm3": round(volume_cm3, 2),
            "size_mm": size_str
        })

    return json_result

In [19]:
import os 
import nibabel as nib 
class_labels = ["EDH","IPH","IVH","SAH","SDH"] 
prediction_path = "/home/tibia/Projet_Hemorragie/MBH_seg_v2_test_1/predictions" 
print(prediction_path) 
print(f"nombre de fichiers : {len(os.listdir(prediction_path))} ")
first_seg_file = os.listdir(prediction_path)[0] 
print(f" premier fichier : {first_seg_file} ")
mask_img = nib.load(os.path.join(prediction_path, first_seg_file))
print(f"shape and type : {mask_img.shape} , {type(mask_img)} ")
mask = mask_img.get_fdata().astype(np.int32)  # array numpy avec les labels
print(f"mask shape and type : {mask.shape} , {type(mask)} ")

spacing = mask_img.header.get_zooms()


json_result = generate_json_per_type(mask, mask_img.header.get_zooms(), class_labels)
print(json_result)

/home/tibia/Projet_Hemorragie/MBH_seg_v2_test_1/predictions
nombre de fichiers : 48 
 premier fichier : ID_0219ef88_ID_e5c1a31210.nii.gz 
shape and type : (512, 512, 31) , <class 'nibabel.nifti1.Nifti1Image'> 
mask shape and type : (512, 512, 31) , <class 'numpy.ndarray'> 
{'EDH': 0, 'IPH': 0, 'IVH': 0, 'SAH': 282, 'SDH': 2520}


{'hemorrhage_detected': True, 'types': [{'type': 'SAH', 'volume_cm3': 0.34, 'size_mm': '63x82x30'}, {'type': 'SDH', 'volume_cm3': 3.06, 'size_mm': '81x148x116'}]}


In [12]:
import nibabel as nib
import numpy as np

def compute_labels_volume(path_segmentation, labels_tag):
    """
    Lit la segmentation une seule fois et retourne un dict {label: volume_mL}.
    - path_segmentation : chemin vers NIfTI (.nii/.nii.gz)
    - labels_tag : liste d'entiers (labels d'intérêt)
    ex: compute_labels_volume_ml(segmentation_patient.nii.gz, [1,2,3])
    """
    img = nib.load(path_segmentation)
    data = img.get_fdata(dtype=np.uint8)
    voxel_vol_mm3 = np.prod(img.header.get_zooms())  # mm per voxel product

    # compter une seule fois tous les labels présents
    labels_present, counts = np.unique(data, return_counts=True)
    counts_map = dict(zip(labels_present.astype(int), counts.astype(int)))

    # construire résultat pour tous les labels demandés (0 si absent)
    volumes_ml = {}
    for lbl in labels_tag:
        c = counts_map.get(int(lbl), 0)
        volumes_ml[int(lbl)] = (int(c) * float(voxel_vol_mm3)) / 1000.0

    return volumes_ml


import nibabel as nib
import numpy as np

def compute_labels_volume_safe(path_segmentation, labels_tag, int_dtype=np.int32, allow_round=False):
    """
    Lit la segmentation une seule fois et retourne un dict {label: volume_mL}.
    - path_segmentation : chemin vers NIfTI (.nii/.nii.gz)
    - labels_tag : iterable d'entiers (labels d'intérêt)
    - int_dtype : dtype entier à utiliser en mémoire (np.int32 recommandé)
    - allow_round : si True, on arrondit les valeurs flottantes avant conversion en entier

    Retour : dict {int(label): float(volume_mL)}
    """
    # chargement
    img = nib.load(path_segmentation)
    voxel_vol_mm3 = float(np.prod(img.header.get_zooms()))  # mm^3 par voxel

    # get_fdata renvoie souvent float64 ; on charge d'abord en float pour vérifier
    if int_dtype is None:
        int_dtype = img.dataobj.dtype  # conserve le type original

    data = np.asanyarray(img.dataobj, dtype=int_dtype)

    # Si data n'est pas entier, soit erreur soit arrondir selon allow_round
    if not np.issubdtype(data.dtype, np.integer):
        # vérifier si toutes les valeurs sont proche d'entiers
        if not np.allclose(data, np.rint(data)):
            if not allow_round:
                raise ValueError(
                    "La segmentation contient des valeurs non entières. "
                    "Si vous voulez autoriser l'arrondi avant conversion, passez allow_round=True."
                )
            # arrondir ensuite convertir
        data = np.rint(data)

    # conversion en dtype entier (copie évitée si possible)
    if isinstance(data, np.ndarray):
        data = data.astype(int_dtype, copy=False, order='C')
    else:
        # pour les objets memmap/dataobj : forcer via np.array (qui accepte copy param)
        data = np.array(data, dtype=int_dtype, copy=False, order='C')

    # compter une seule fois tous les labels présents
    labels_present, counts = np.unique(data, return_counts=True)
    counts_map = dict(zip(labels_present.astype(int), counts.astype(int)))

    # construire résultat (0 si absent)
    volumes_ml = {}
    for lbl in labels_tag:
        lbl_int = int(lbl)
        c = counts_map.get(lbl_int, 0)
        volumes_ml[lbl_int] = (int(c) * voxel_vol_mm3) / 1000.0  # mm^3 -> mL

    return volumes_ml


In [20]:
path_segmentation = os.path.join(prediction_path, first_seg_file)
print(path_segmentation)
compute_labels_volume_safe(path_segmentation , [1,2,3,4,5])

/home/tibia/Projet_Hemorragie/MBH_seg_v2_test_1/predictions/ID_0219ef88_ID_e5c1a31210.nii.gz


{1: 0.0, 2: 0.0, 3: 0.0, 4: 0.341885728597641, 5: 3.0551490640640258}